In [1]:
import os

import pandas as pd

import datasets as hfd

In [2]:
DATA_ROOT_DP = os.environ['DATA_HOME']
CV_DP = f'{DATA_ROOT_DP}/datasets/cv-corpus-8.0-2022-01-19/be'

In [3]:
os.listdir(CV_DP)

['dev.tsv',
 'validated.tsv',
 'train.tsv',
 'clips',
 'reported.tsv',
 'invalidated.tsv',
 'other.tsv',
 'test.tsv']

In [4]:
df_train = pd.read_csv(os.path.join(CV_DP, 'train.tsv'), delimiter='\t')
df_dev = pd.read_csv(os.path.join(CV_DP, 'dev.tsv'), delimiter='\t')
df_test = pd.read_csv(os.path.join(CV_DP, 'test.tsv'), delimiter='\t')
df_other = pd.read_csv(os.path.join(CV_DP, 'other.tsv'), delimiter='\t')

print(f'df_train.shape: {df_train.shape}')
print(f'df_dev.shape: {df_dev.shape}')
print(f'df_test.shape: {df_test.shape}')
print(f'df_other.shape: {df_other.shape}')

df_train.shape: (314305, 10)
df_dev.shape: (15803, 10)
df_test.shape: (15801, 10)
df_other.shape: (40596, 10)


## combine sentences from df_train and df_other to build LM

In [5]:
s_from_other = set(df_other['sentence']).difference(df_dev['sentence']).difference(df_test['sentence'])
print(len(s_from_other))

26906


In [6]:
s_train_other = list(set(df_train['sentence']).union(s_from_other))
print(f'len(s_train_other): {len(s_train_other)}')

len(s_train_other): 314676


In [7]:
df = pd.DataFrame({'sentence': s_train_other})
df.loc[df['sentence'].isin(df_train['sentence']), 'split'] = 'train'
df.loc[~df['sentence'].isin(df_train['sentence']), 'split'] = 'other'
print(f'df.shape: {df.shape}')
df['split'].value_counts(dropna=False)

df.shape: (314676, 2)


train    314305
other       371
Name: split, dtype: int64

## save

In [10]:
fp = 'artifacts/lm/sentences_for_lm.tsv'
os.makedirs(os.path.dirname(fp), exist_ok=True)

In [12]:
df.to_csv(fp, index=False, sep='\t')

In [14]:
!head -10 $fp

sentence	split
Вы ўжо ўсе, відавочна, адчулі, што нешта не так?	train
Зойдзе да яе, а яна не ведае, з чаго пачынаць гаворку.	train
І трэба істотна прызавыя і іншае дадаваць за вынік на ўнутраным чэмпіянаце.	train
Каб дзевяць кампазіцый на канцэрце гучалі амаль як на пласцінцы.	train
Але маюцца межавыя ўмовы.	train
Гэта можа быць пералом.	train
Сапраўды, цяпер гістарычны помнік выглядае інакш, чым месяц таму.	train
І вось прыехаў да вас.	train
Апроч таго, мы яшчэ не абнавілі свой авіяцыйны парк.	train


In [15]:
!wc $fp

  314677  2700184 30778659 artifacts/lm/sentences_for_lm.tsv
